In [1]:
import numpy as np
import math as mt
from itertools import combinations
from random import randint, random

In [2]:
rad = 180./mt.pi
def mag(x):
    return np.sqrt(np.dot(x,x))
def cosang(x,y):
    return np.dot(x,y)/(mag(x)*mag(y))
def averg(xlist):
    return sum(np.array(xlist))/len(xlist)
def deg2rad(cell):
    return cell*np.array([1,1,1,1./rad, 1./rad, 1./rad])
def rad2deg(cell):
    return cell*np.array([1,1,1,rad, rad, rad])

In [3]:
def basis2cell(basis):
    """Takes crystal basis as columns of a matrix"""
    tbasis = np.transpose(basis)
    lengths = [mag(i) for i in tbasis]
    cosangs = [cosang(tbasis[i],tbasis[j]) for i,j in 
               [[1,2],[0,2],[0,1]]]
    angs = [np.arccos(i) for i in cosangs]
    lengths.extend(angs)
    return lengths

In [4]:
def dcell2dbasis(a,b,c,alpha,beta,gamma):
    """Returns basis as rows of a matrix."""
    temp = (np.cos(alpha)-np.cos(beta)*np.cos(gamma))/np.sin(gamma)
    return [[a,b*np.cos(gamma),c*np.cos(beta)],
            [0,b*np.sin(gamma),c*temp],
           [0,0,c*np.sqrt(1-(np.cos(beta)**2)-temp**2)]]

In [5]:
def rcell2rbasis(a,b,c,alpha,beta,gamma):
    """Returns reciprocal lattice as columns of matrix"""
    temp = (np.cos(gamma)-np.cos(beta)*np.cos(alpha))/np.sin(alpha)
    return [[a*np.sqrt(1-(np.cos(beta)**2))-temp**2, 0, 0],
            [a*temp, b*np.sin(alpha),0],
            [a*np.cos(beta), b*np.cos(alpha), c]]

In [6]:
def dspace(hkl, dcell):
    """Returns the d-spacing of a RL point given cell params"""
    dbasis = dcell2dbasis(dcell)
    gmat = np.matmul(np.transpose(dbasis), dbasis)
    gmatstar = np.linalg.inv(gmat)
    return 1./np.sqrt(np.matmul(np.matmul(hkl,gmatstar),hkl))

In [7]:
def basis2basis(basis):
    return np.linalg.inv(np.transpose(basis))

In [8]:
def distance(pos1, pos2, dcell):
    """Measures the separation given two lattice-coordinate 
    positions and the cell parameters."""
    dbasis = dcell2dbasis(*dcell)
    gmat = np.matmul(np.transpose(dbasis),dbasis)
    diff = np.array(pos2)-np.array(pos1)
    return np.sqrt(np.dot(diff,np.matmul(gmat,diff)))

In [9]:
a=[[-3.23277, -3.92086, -0.947768], 
   [-1.07693, 3.13812, 4.91741], 
   [2.73895, 1.08678, -1.2721]]

In [10]:
dcell0 = [5.00, 5.05, 4.97, mt.pi/2. - 0.025, mt.pi/2. + 0.012, mt.pi/2. + 0.018 ]

In [11]:
randrot = [[-0.739464, -0.246336, 0.626508], 
           [0.428713, -0.889847, 0.15613], 
            [0.519036, 0.384045, 0.763617]]
np.round(np.matmul(np.transpose(randrot),randrot))

array([[ 1.,  0., -0.],
       [ 0.,  1.,  0.],
       [-0.,  0.,  1.]])

In [12]:
dbasis0 = np.matmul(randrot, dcell2dbasis(*dcell0))

In [13]:
# dbasis0 is now a random cell with a random orientation 
# in the plane.
# First we want to extract it's parameters
dcell1 = basis2cell(dbasis0)
dcell1

[5.0000005324024714,
 5.0499990946308815,
 4.9709117226840673,
 1.5458009148872118,
 1.5827945446888554,
 1.5887955243261072]

In [14]:
# Now we find the basis in the canonical orientation
dbasis1 = dcell2dbasis(*dcell1)
dbasis1

[[5.0000005324024714, -0.090891023373546917, -0.059640651005852342],
 [0, 5.0491810898048444, 0.12318357603766673],
 [0, 0, 4.969939835523963]]

In [15]:
# find the rotation from original
rotation = np.round(np.matmul(dbasis0, np.linalg.inv(dbasis1)),6)
validate = np.matmul(rotation, np.transpose(rotation))
print(rotation)
print(validate)

[[-0.739464 -0.246335  0.626393]
 [ 0.428713 -0.889847  0.156102]
 [ 0.519036  0.384045  0.763477]]
[[  9.99856130e-01  -3.61690010e-05  -1.75513318e-04]
 [ -3.61690010e-05   9.99990354e-01  -4.35237930e-05]
 [ -1.75513318e-04  -4.35237930e-05   9.99786061e-01]]


In [16]:
# symmetrize the cell parameters (this assumes a cubic lattice)
alatt = np.mean(dcell1[:3])
alatt

5.0069704499058068

In [17]:
alpha = mt.pi/2.

In [18]:
dcell2 = [alatt,alatt,alatt,alpha,alpha,alpha]

In [19]:
dbasis2 = dcell2dbasis(*dcell2)
dbasis2

[[5.0069704499058068, 3.0658851674512647e-16, 3.0658851674512647e-16],
 [0, 5.0069704499058068, 3.0658851674512647e-16],
 [0, 0, 5.0069704499058068]]

In [20]:
dbasis3 = np.matmul(rotation,dbasis2)
dbasis3

array([[-3.7024744 , -1.23339207,  3.13633124],
       [ 2.14655332, -4.45543763,  0.7815981 ],
       [ 2.59879791,  1.92290197,  3.82270678]])

In [21]:
dbasis3-dbasis0

array([[-0.0051544 , -0.05681044,  0.00943952],
       [ 0.00298832,  0.07652966,  0.14096615],
       [ 0.00361791,  0.03096672,  0.01191917]])

In [22]:
import pyniggli

In [23]:
temp = pyniggli.reduced_cell(dbasis3)

In [24]:
temp._get_params()

(25.060556357165929,
 25.069758425109644,
 25.069732470344139,
 3.4169572113995628e-06,
 5.4359247041446679e-06,
 -3.4322497484140513e-06,
 0,
 0,
 0)

In [9]:
def symmetrize(cell_params, case):
    a0, b0, c0, alpha0, beta0, gamma0 = cell_params
    if case == 1:
        a =b = c = np.mean(cell_params[:3])
        alpha = beta = gamma = mt.pi/3.
    elif case == 2 or case == 4:
        a =b = c = np.mean(cell_params[:3])
        alpha = beta = gamma = np.mean(cell_params[3:])
    elif case == 3:
        a =b = c = np.mean(cell_params[:3])
        alpha = beta = gamma = mt.pi/2.
    elif case == 5:
        a =b = c = np.mean(cell_params[:3])
        alpha = beta = gamma = 1.910633236249019
    elif case == 6:
        a =b = c = np.mean(cell_params[:3])
        alpha = beta = np.arccos(((((a**2+b**2)/2.)-np.cos(gamma0)*a*c)/2.)/(b*c)) 
        gamma = cell_params[5]
    elif case == 7:
        a = b = c = np.mean(cell_params[:3])
        alpha = np.arccos(((((a**2+b**2)/2.)-2*np.cos(gamma0)*a*c))/(b*c)) 
        beta = gamma = np.mean([beta0, gamma0])
    elif case == 8:
        a =b = c = np.mean(cell_params[:3])
        alpha = np.arccos((((a**2+b**2)/2.)-np.cos(gamma0)*a*c-np.cos(beta0)*a*b)/(b*c))
        beta = beta0
        gamma = gamma0
    elif case == 9:
        a =b = np.mean([a0,b0])
        c = c0
        alpha = np.arccos((a*a)/(2*b*c))
        beta = np.arccos((a*a)/(2*a*c))
        gamma = mt.pi/3
    elif case == 10:
        a =b = np.mean([a0,b0])
        c = c0
        alpha = beta = np.mean([alpha0, beta0])
        gamma = gamma0
    elif case == 11:
        a =b = np.mean([a0,b0])
        c = c0
        alpha = beta = gamma = mt.pi/2.
    elif case == 12:
        a =b = np.mean([a0,b0])
        c = c0
        alpha = beta = mt.pi/2.
        gamma = 2*mt.pi/3.
    elif case == 13:
        a =b = np.mean([a0,b0])
        c = c0
        alpha = beta = mt.pi/2.
        gamma = gamma0
    elif case == 15:
        a =b = np.mean([a0,b0])
        c = c0
        alpha = np.arccos(-(a*a)/(2*b*c))
        beta = np.arccos(-(a*a)/(2*a*c))
        gamma = mt.pi/2.
    elif case == 16:
        a =b = np.mean([a0,b0])
        c = c0
        gamma = mt.pi/2.
        alpha = np.arccos(((((a**2+b**2)/2.)-np.cos(gamma)*a*c)/2.)/(b*c))
        beta = np.arccos(((((a**2+b**2)/2.)-np.cos(gamma)*a*c)/2.)/(a*c))
    elif case == 14:
        a =b = np.mean([a0,b0])
        c = c0
        alpha = beta = np.mean([alpha0,beta0])
        gamma = gamma0
    elif case == 17:
        a =b = np.mean([a0,b0])
        c = c0
        alpha = np.arccos((((a**2+b**2)/2.)-np.cos(gamma0)*a*c-np.cos(beta0)*a*b)/(b*c))
        beta = beta0
        gamma = gamma0
    elif case == 18:
        a = a0
        b = c = np.mean([c0,b0])
        alpha = np.arccos((a*a)/(4*b*c))
        beta = np.arccos((a*a)/(2*a*c))
        gamma = np.arccos((a*a)/(2*a*b))
    elif case == 19:
        a = a0
        b = c = np.mean([c0,b0])
        alpha = alpha0
        beta = np.arccos((a*a)/(2*a*c))
        gamma = np.arccos((a*a)/(2*a*b))
    elif case == 20 or case == 25:
        a = a0
        b = c = np.mean([c0,b0])
        alpha = alpha0
        beta = gamma = np.mean([beta0,gamma0])
    elif case == 21:
        a = a0
        b = c = np.mean([c0,b0])
        alpha = beta = gamma = mt.pi/2
    elif case == 22:
        a = a0
        b = c = np.mean([c0,b0])
        alpha = np.arccos(-(b*b)/(2*b*c))
        beta = gamma = mt.pi/2.
    elif case == 23:
        a = a0
        b = c = np.mean([c0,b0])
        alpha = alpha0
        beta = gamma = mt.pi/2.
    elif case == 24:
        a = a0
        b = c = np.mean([c0,b0])
        beta = np.arccos(-(a*a)/(3*a*c))
        gamma = np.arccos(-(a*a)/(3*a*b))
        alpha = np.arccos((((a**2+b**2)/2.)-np.cos(gamma)*a*c-np.cos(beta)*a*b)/(b*c))
    elif case == 26:
        a = a0
        b = b0
        c = c0
        alpha = np.arccos((a*a)/(4*b*c))
        beta = np.arccos((a*a)/(2*a*c))
        gamma = np.arccos((a*a)/(2*a*b))
    elif case == 27:
        a = a0
        b = b0
        c = c0
        alpha = alpha0
        beta = np.arccos((a*a)/(2*a*c))
        gamma = np.arccos((a*a)/(2*a*b))
    elif case == 28:
        a = a0
        b = b0
        c = c0
        alpha = alpha0
        beta = np.arccos((a*a)/(2*a*c))
        gamma = np.arccos(2*np.cos(alpha)*b*c/(a*b))
    elif case == 29:
        a = a0
        b = b0
        c = c0
        alpha = alpha0
        beta = np.arccos(2*np.cos(alpha)*b*c/(a*c))
        gamma = np.arccos((a*a)/(2*a*b))
    elif case == 30:
        a = a0
        b = b0
        c = c0
        alpha = np.arccos((b*b)/(2*b*c))
        beta = beta0
        gamma = np.arccos(2*np.cos(beta)*a*c/(a*b))
    elif case == 31 or case == 44:
        a = a0
        b = b0
        c = c0
        alpha = alpha0
        beta = beta0
        gamma = gamma0
    elif case == 32:
        a = a0
        b = b0
        c = c0
        alpha = beta = gamma = mt.pi/2.
    elif case == 40:
        a = a0
        b = b0
        c = c0
        alpha = np.arccos(-(b*b)/(2*b*c))
        beta = gamma = mt.pi/2.
    elif case == 35:
        a = a0
        b = b0
        c = c0
        alpha = alpha0
        beta = gamma = mt.pi/2.
    elif case == 36:
        a = a0
        b = b0
        c = c0
        alpha = gamma = mt.pi/2.
        beta = np.arccos(-(a*a)/(2*a*c))
    elif case == 33:
        a = a0
        b = b0
        c = c0
        alpha = gamma = mt.pi/2.
        beta = beta0
    elif case == 38:
        a = a0
        b = b0
        c = c0
        alpha = beta = mt.pi/2.
        gamma = np.arccos(-(a*a)/(2*a*b))
    elif case == 34:
        a = a0
        b = b0
        c = c0
        alpha = beta = mt.pi/2.
        gamma = gamma0
    elif case == 42:
        a = a0
        b = b0
        c = c0
        alpha = np.arccos(-(b*b)/(2*b*c))
        beta = np.arccos(-(a*a)/(2*a*c))
        gamma = mt.pi/2.
    elif case == 41:
        a = a0
        b = b0
        c = c0
        alpha = np.arccos(-(b*b)/(2*b*c))
        beta = beta0
        gamma = mt.pi/2.
    elif case == 37:
        a = a0
        b = b0
        c = c0
        alpha = alpha0
        beta = np.arccos(-(a*a)/(2*a*c))
        gamma = mt.pi/2.
    elif case == 39:
        a = a0
        b = b0
        c = c0
        alpha = alpha0
        beta = mt.pi/2.
        gamma = np.arccos(-(a*a)/(2*a*b))
    elif case == 43:
        a = a0
        b = b0
        c = c0
        D = ((a**2+b**2)/2.)-np.cos(gamma)*a*c-np.cos(beta)*a*b
        if np.abs(2*D+np.cos(gamma0)*a*b)==b*b:
            alpha = np.arccos(D/(b*c))
            beta = beta0
            gamma = gamma0
        else:
            D = (-(a**2+b**2)/2.)-np.cos(gamma)*a*c-np.cos(beta)*a*b
            alpha = np.arccos(D/(b*c))
            beta = beta0
            gamma = gamma0
    else:
        raise IOError("Case {} not found".format(case))

    return [a, b, c, alpha, beta, gamma]

In [10]:
def delinter(obasis,case):
    """Takes the original basis as columns of a matrix and the 
    niggli case number. Returns the 'corrected' basis."""
    ocell_params = basis2cell(obasis)
    canonical_basis = dcell2dbasis(*ocell_params)
    rot = np.matmul(obasis, np.linalg.inv(canonical_basis))
    val = np.round(np.matmul(rot, np.transpose(rot)),3)
    if not np.allclose(val, [[1,0,0],[0,1,0],[0,0,1]],1E-3):
        print(val)
        raise ValueError("Rotation matrix in delinter "
                        "is not correct.")
    canonical_params = symmetrize(ocell_params, case)
    fixed_canonical_basis = dcell2dbasis(*canonical_params)
    fixed_basis = np.matmul(rot,fixed_canonical_basis)
    
    return fixed_basis
    
    
    

In [11]:
A = np.transpose([[  6.99989831031789,   0.00001427664054,   0.],
                  [3.49996141765365,   6.06195444738213,   0.0],
                  [0.0,   0.0,   4.74600387583220]])

In [13]:
from opf_python.niggli_lat_id import niggli_id
from pyniggli import reduced_cell

In [14]:
(typ, case, syms, An) = niggli_id(A)
temp = reduced_cell(A)
An = temp.niggli
mat = np.dot(temp.original,temp.C)
print(mat[:,0])
print(np.dot(mat[:,0],mat[:,0]))

('mat', array([[  6.99989831e+00,   3.49996142e+00,   0.00000000e+00],
       [  1.42766405e-05,   6.06195445e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   4.74600388e+00]]))
('mat', array([[ -3.49996142e+00,  -6.99989831e+00,   0.00000000e+00],
       [ -6.06195445e+00,  -1.42766405e-05,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,  -4.74600388e+00]]))
('mat', array([[  3.49996142e+00,   0.00000000e+00,   6.99989831e+00],
       [  6.06195445e+00,   0.00000000e+00,   1.42766405e-05],
       [  0.00000000e+00,   4.74600388e+00,   0.00000000e+00]]))
('mat', array([[  0.00000000e+00,  -3.49996142e+00,  -6.99989831e+00],
       [  0.00000000e+00,  -6.06195445e+00,  -1.42766405e-05],
       [ -4.74600388e+00,   0.00000000e+00,   0.00000000e+00]]))
('mat', array([[  0.00000000e+00,  -3.49996142e+00,   6.99989831e+00],
       [  0.00000000e+00,  -6.06195445e+00,   1.42766405e-05],
       [  4.74600388e+00,   0.00000000e+00,   0.00000000e+00]]))
('mat

In [146]:
case

23

In [15]:
Ad = delinter(An, case)

In [170]:
niggli_id(Ad)

('base centered orthorhombic',
 23,
 5,
 array([[-0.3333333,  1.       ,  2.       ],
        [-1.54116  ,  1.       , -1.       ],
        [ 1.87449  ,  1.       , -1.       ]]))

In [171]:
Ad-An

array([[  0.00000000e+00,  -6.15674066e-06,   6.15676214e-06],
       [  0.00000000e+00,   1.06635618e-05,   1.06635494e-05],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [172]:
np.matmul(Ad,np.linalg.inv(temp.C))-A

array([[ -2.14805951e-11,  -6.15676214e-06,   0.00000000e+00],
       [ -2.13271113e-05,  -1.06635494e-05,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [174]:
np.round(np.matmul(Ad,np.linalg.inv(temp.C)),6)

array([[  6.99989800e+00,   3.49995500e+00,   0.00000000e+00],
       [ -7.00000000e-06,   6.06194400e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   4.74600400e+00]])

In [18]:
np.matmul(Ad,np.linalg.inv(temp.C))

array([[  6.99989831e+00,   3.49995526e+00,   0.00000000e+00],
       [ -7.05047076e-06,   6.06194378e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   4.74600388e+00]])

In [180]:
Ad

array([[ 0.        , -3.49994305, -3.49995526],
       [ 0.        ,  6.06195083, -6.06194378],
       [ 4.74600388,  0.        ,  0.        ]])

In [181]:
A

array([[  6.99989831e+00,   3.49996142e+00,   0.00000000e+00],
       [  1.42766405e-05,   6.06195445e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   4.74600388e+00]])

In [132]:
np.allclose(np.matmul(An,np.linalg.inv(temp.C)), A)

True

In [177]:
np.linalg.det(np.linalg.inv(temp.C))

1.0

In [178]:
temp.C

array([[ 0., -1.,  0.],
       [ 0.,  1., -1.],
       [ 1.,  0.,  0.]])

In [185]:
niggli_id(Ad,eps_=1E-1)

('hexagonal', 22, 2, array([[ 0.       ,  1.       , -0.5      ],
        [ 0.       ,  0.       ,  0.8660254],
        [-0.5      ,  0.       ,  0.       ]]))

In [19]:
Ad

array([[ 0.        , -3.49994305, -3.49995526],
       [ 0.        ,  6.06195083, -6.06194378],
       [ 4.74600388,  0.        ,  0.        ]])

In [2]:
import numpy as np

In [5]:
np.round(np.pi,18)

3.1415926535897931

In [11]:
any([i in ["KPDENSITY","KPPRA"] for i in {"NKPTS":"3000"}.keys()])

False